In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [2]:
# Cuando se inicia la página de glassdoor, en la caja de texto por defecto me aparece mi localización actual
#por lo que tengo que limpiarla antes de poder meter el valor que busco.

def clearText(element):
    '''Clear the text box for location value.'''
    length = len(element.get_attribute('value'))
    element.send_keys(length * Keys.BACKSPACE)

In [33]:
def get_jobs(keyword, location, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
        # Initializing the webdriver.
    options = webdriver.ChromeOptions()

        # Open a private Chrome tab in case we want it to.
    #options.add_argument("--incognito")
    
        #Disable notifications.
    #options.add_argument('--disable-notifications')
    
        # Installing webdriver for each session in case it needs an update.
    driver = webdriver.Chrome(ChromeDriverManager().install(), options = options)
    driver.set_window_size(1120, 1000)

    loginUrl = 'https://www.glassdoor.es/index.htm'
    driver.get(loginUrl)

    jobs = []

        # Pressing login button    
    driver.find_element_by_xpath('//*[@id="TopNav"]/nav/div/div/div[4]/div[1]/a').click()
 
        # Defining username and password.
    usname = 'borcoferpa92@gmail.com'
    pwd = 'y3t5XepkKc7agPL'

        # Log into my accoun by writing username, password and clicking login.
    driver.find_element_by_xpath('//*[@id="userEmail"]').send_keys(usname)
    driver.find_element_by_xpath('//*[@id="userPassword"]').send_keys(pwd)
    driver.find_element_by_xpath('//*[@id="LoginModal"]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[3]/div[1]/button').click()

    time.sleep(5)

        # Select cookies preferences. Only the needed.
    try:
        driver.find_element_by_xpath('//*[@id="onetrust-pc-btn-handler"]').click()
        driver.find_element_by_xpath('//*[@id="cookie-preferences"]/div[5]/button').click()
    except NoSuchElementException:
        pass

    time.sleep(2)

        # Writes the keyword in keyword box. The job position we are looking for.
    driver.find_element_by_xpath('//*[@id="sc.keyword"]').send_keys(keyword)

    time.sleep(2)

        # We use the previuos function to clear the text box.
    clearText(driver.find_element_by_xpath('//*[@id="sc.location"]'))

    time.sleep(2)
                                   
        # Writes de location where we want to find a job position.
    driver.find_element_by_xpath('//*[@id="sc.location"]').send_keys(location)

        # Click search button.
    driver.find_element_by_xpath('//*[@id="scBar"]/div/button').click()

        #Close the popup.
    try:
        driver.find_element_by_xpath('//*[@id="prefix__icon-close-1"]').click()
    except NoSuchElementException:
        pass

    time.sleep(2)

    # If true should be looking for new jobs.
    while len(jobs) < num_jobs:  
        
        #jl for Job Listing. These are the buttons we're going to click where job information is.
        job_buttons = driver.find_elements_by_class_name('react-job-listing') 
        
        time.sleep(5)
        
        try:
            # Closing the modal.
            driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
        except NoSuchElementException:
            pass
        
        for job_button in job_buttons: 
            
            try:
                # Closing the modal.
                driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            except NoSuchElementException:
                pass

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
        
            if len(jobs) >= num_jobs:
                break

            job_button.click()
            
            time.sleep(5)
            
            collected_successfully = False
            
            # Collecting information we want.
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]').text
                    location = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[3]').text
                    job_title = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[2]').text
                    collected_successfully = True
                except:
                    time.sleep(10)
            
            try:
                rating = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[1]/span').text
            except NoSuchElementException:
                rating = 0 #You need to set a "not found value". It's important.

            # Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

                 # Adding job to jobs.
            jobs.append({"Job Title" : job_title,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location})

            # Clicking on the "next page" button .                
        if len(jobs) < num_jobs:
            try:
                driver.find_element_by_xpath('//*[@id="FooterPageNav"]/div/ul/li[7]/a/span/svg').click()
            except NoSuchElementException:
                print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
                break

    # This line converts the dictionary object into a pandas DataFrame.
    return pd.DataFrame(jobs)



In [34]:
# Getting data.
df = get_jobs('cibersecurity', 'madrid', 5, True)

# Saving dataframe to csv. We have to change the file name everytime we activate the function.
df.to_csv('/Users/BCFP/Downloads/Glassdoor scraping/cibersecurity.csv', index = None, header = True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [/Users/BCFP/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache




Progress: 0/5
Job Title: Cibersecurity Engineer
Rating: 4.1
Company Name: Deimos Space
4.1
Location: Madrid
Progress: 1/5
Job Title: Internship Program to STEM profiles with Disabilty
Rating: 0
Company Name: Fundación Universia
Location: Madrid
Progress: 2/5
Job Title: Cybersecurity engineer
Rating: 3.8
Company Name: GMV
3.8
Location: Tres Cantos
Scraping terminated before reaching target number of jobs. Needed 5, got 3.


In [35]:
pd.read_csv('cibersecurity.csv')

,Job Title,Rating,Company Name,Location
0,Cibersecurity Engineer,4.1,Deimos Space\n4.1,Madrid
1,Internship Program to STEM profiles with Disab...,0.0,Fundación Universia,Madrid
2,Cybersecurity engineer,3.8,GMV\n3.8,Tres Cantos
